# Pre-processing of Data

1. Extracting the raw questions, options, answers from the .js file
2. Splitting the data to use for research
3. Creation of the test cases:
    - Case 1: Wrong answer, wrong explanation
    - Case 2: Wrong answer, right explanation
    - Case 3: Right answer, wrong explanation
    - Case 4: Right answer, right explanation

## 1. Extracting the raw questions, options, answers

In [7]:
import json
import pandas as pd

def clean_js_content(content):
    # Remove 'const' declarations and combine arrays
    content = content.replace('const', '')
    
    # Split the content into individual array declarations
    arrays = ['easy_arrays', 'medium_arrays', 'hard_arrays', 
              'easy_stacksandqueues', 'medium_stacksandqueues', 'hard_stacksandqueues',
              'easy_linkedlist', 'medium_linkedlist', 'hard_linkedlists',
              'easy_recursion', 'medium_recursion', 'hard_recursion',
              'easy_trees', 'medium_trees', 'hard_trees',
              'easy_hashing', 'medium_hashing', 'hard_hashing',
              'easy_heaps', 'medium_heaps', 'hard_heaps',
              'easy_graphs', 'medium_graphs', 'hard_graphs']
    
    all_questions = []
    
    for array_name in arrays:
        try:
            # Find the start of the array
            start_idx = content.find(array_name + ' = [')
            if start_idx == -1:
                continue
                
            # Find the end of the array
            end_idx = content.find('\n]', start_idx)
            if end_idx == -1:
                continue
                
            # Extract the array content
            array_content = content[start_idx + len(array_name + ' = ['):end_idx + 1]
            
            # Convert the content to valid JSON format
            array_content = '[' + array_content.strip()
            array_content = array_content.replace('\n', '')
            array_content = array_content.replace('    ', '')
            
            # Add missing commas between objects
            array_content = array_content.replace('}{', '},{')
            
            # Add missing brackets if needed
            if not array_content.startswith('['):
                array_content = '[' + array_content
            if not array_content.endswith(']'):
                array_content = array_content + ']'
            
            # Fix missing commas between objects
            array_content = array_content.replace('"}{"', '"},{"')
            
            try:
                questions = json.loads(array_content)
                all_questions.extend(questions)
            except json.JSONDecodeError as e:
                print(f"Error parsing {array_name}: {e}")
                continue
            
        except Exception as e:
            print(f"Error processing {array_name}: {e}")
            continue
    return all_questions

# Read the JS file
with open('./data/questions.js', 'r') as file:
    content = file.read()

# Combine all questions into one list
combined_questions = clean_js_content(content)

# Now create the DataFrame
df = pd.DataFrame({
    'Title': [q['title'] for q in combined_questions],
    'Question': [q['question'] for q in combined_questions],
    'Options': [q['options'] for q in combined_questions],
    'Answer': [q['options'][q['ans']] for q in combined_questions]
})

print(f"Total number of questions: {len(df)}")
print(df.head())

Total number of questions: 240
         Title                                           Question  \
0  Easy Arrays                                  What is an array?   
1  Easy Arrays                How is memory allocated for arrays?   
2  Easy Arrays       What does the Insert operation do in arrays?   
3  Easy Arrays        How are Python lists different from arrays?   
4  Easy Arrays  What is the purpose of initializing the size o...   

                                             Options  \
0  [A collection of similar elements, A collectio...   
1  [Memory is allocated randomly, Memory is alloc...   
2  [Deletes an element, Searches for an element, ...   
3  [Python lists do not store values, Python list...   
4  [To allocate memory, To delete elements, To se...   

                                        Answer  
0             A collection of similar elements  
1         Memory is allocated at the beginning  
2                        Inserts a new element  
3  Python lists can s

In [45]:
df.to_excel("questions.xlsx", index=False)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     240 non-null    object
 1   Question  240 non-null    object
 2   Options   240 non-null    object
 3   Answer    240 non-null    object
dtypes: object(4)
memory usage: 7.6+ KB


## 2. Splitting the data to use for research

In [9]:
# Extracting first 5 rows of each Title for testing

first_per_title = df.groupby('Title').head(1)

# Sort by Title to keep it organized
first_per_title = first_per_title.sort_values('Title')

# Reset the index
first_per_title = first_per_title.reset_index(drop=True)

first_per_title['Title'].value_counts()
# 

Title
Easy Arrays                 1
Easy Graphs                 1
Medium Stacks and Queues    1
Medium Recursion            1
Medium Linked Lists         1
Medium Heaps                1
Medium Hashing              1
Medium Graphs               1
Medium Arrays               1
Hard Trees                  1
Hard Stacks and Queues      1
Hard Recursion              1
Hard Linked Lists           1
Hard Heaps                  1
Hard Hashing                1
Hard Graphs                 1
Hard Arrays                 1
Easy Trees                  1
Easy Stacks and Queues      1
Easy Recursion              1
Easy Linked Lists           1
Easy Heaps                  1
Easy Hashing                1
Medium Trees                1
Name: count, dtype: int64

In [10]:
first_per_title.head()

,Title,Question,Options,Answer
0,Easy Arrays,What is an array?,"[A collection of similar elements, A collectio...",A collection of similar elements
1,Easy Graphs,What is a minimum spanning tree?,[A tree with the minimum number of edges from ...,A tree that minimizes the total weight of edge...
2,Easy Hashing,What is the primary purpose of using hashing i...,"[To sort data efficiently, To store data in a ...",To quickly retrieve data based on a key
3,Easy Heaps,What data structure are heaps almost always im...,"[Linked lists, Arrays, Hash tables, Stacks]",Arrays
4,Easy Linked Lists,What is the main advantage of using a linked l...,"[Constant time access to elements, Contiguous ...",Dynamic size


## 3. Creation of the test cases

In [11]:
from tqdm.notebook import tqdm
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def generate_explanations(df, retriever, model):
    """
    Generate explanations for a DataFrame of questions.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame with columns 'Question', 'Options', 'Answer'
    retriever : Retriever
        LangChain retriever for context
    model : ChatModel
        Language model for generating explanations
    prompt_template : str
        Prompt template for explanation generation
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with added 'Explanation' column
    """
    template = """
    Answer the question based only on the following context:

    {context}

    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    def format_docs(docs):
        return "\n\n".join([d.page_content for d in docs])

    # Create the chain
    # chain = (
    #     {"context": retriever | format_docs, "question": RunnablePassthrough()}
    #     | prompt
    #     | model
    #     | StrOutputParser()
    # )

    chain = (
        RunnablePassthrough()
        | model
        | StrOutputParser()
    )

    # Function to generate explanation for a single row
    def generate_explanation(row):
        text = f"""
        Given the question and the options:
        {row['Question']}
        {row['Options']}

        The correct answer is {row['Answer']}.

        Please do the following:
        1. Give an accurate explanation for the correct answer
        2. Choose one of the wrong answers
        3. Pretend you are a misguided student, create a plausible wrong explanation for the chosen wrong answer

        ## Sample output format in JSON respectively:
        "Correct explanation": "XXX", "Wrong chosen answer": "YYY", "Wrong explanation": "ZZZ"
        """
        
        try:
            explanation = chain.invoke(text)
            return explanation
        except Exception as e:
            print(f"Error generating explanation: {e}")
            return None

    # Apply the explanation generation to each row
    tqdm.pandas()
    df['Explanation'] = df.apply(generate_explanation, axis=1)
    
    return df

In [13]:
import os
from dotenv import load_dotenv
from langchain_community.vectorstores.faiss import FAISS
from langchain_openai import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings

load_dotenv()

model = AzureChatOpenAI(
    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
    api_key=os.environ['AZURE_OPENAI_API_KEY'],
    deployment_name=os.environ['AZURE_OPENAI_DEPLOYMENT_NAME'],
    model_name=os.environ['AZURE_OPENAI_MODEL_NAME'],
    api_version=os.environ['AZURE_OPENAI_API_VERSION'],
    temperature=0
)

embedding_model = AzureOpenAIEmbeddings(azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'], 
                                   api_key=os.environ['AZURE_OPENAI_API_KEY'], 
                                   model=os.environ['TEXT_EMBEDDING_MODEL_NAME'],
                                   azure_deployment=os.environ['TEXT_EMBEDDING_DEPLOYMENT_NAME'])
docsearch = FAISS.load_local(folder_path="./embed", embeddings=embedding_model, allow_dangerous_deserialization=True)
retriever = docsearch.as_retriever()

generate_explanations(first_per_title, retriever, model)

,Title,Question,Options,Answer,Explanation
0,Easy Arrays,What is an array?,"[A collection of similar elements, A collectio...",A collection of similar elements,"{\n ""Correct explanation"": ""An array is a dat..."
1,Easy Graphs,What is a minimum spanning tree?,[A tree with the minimum number of edges from ...,A tree that minimizes the total weight of edge...,"{\n ""Correct explanation"": ""A minimum spannin..."
2,Easy Hashing,What is the primary purpose of using hashing i...,"[To sort data efficiently, To store data in a ...",To quickly retrieve data based on a key,"{\n ""Correct explanation"": ""Hashing is used i..."
3,Easy Heaps,What data structure are heaps almost always im...,"[Linked lists, Arrays, Hash tables, Stacks]",Arrays,"{\n ""Correct explanation"": ""Heaps are almost ..."
4,Easy Linked Lists,What is the main advantage of using a linked l...,"[Constant time access to elements, Contiguous ...",Dynamic size,"{\n ""Correct explanation"": ""The main advantag..."
5,Easy Recursion,What is a characteristic of recursive routines?,"[They call themselves., Each call performs its...",They call themselves.,"{\n ""Correct explanation"": ""A characteristic ..."
6,Easy Stacks and Queues,What data structure follows the Last In First ...,"[Queue, Stack, Linked List, Array]",Stack,"{\n ""Correct explanation"": ""A Stack is a data..."
7,Easy Trees,"In a binary tree, what is the maximum number o...","[0, 1, 2, 3]",2,"{\n ""Correct explanation"": ""In a binary tree,..."
8,Hard Arrays,Explain the difference between an unordered ar...,[Unordered arrays have faster search operation...,Ordered arrays store elements in ascending or ...,"{\n ""Correct explanation"": ""Ordered arrays st..."
9,Hard Graphs,Explain the difference between a strong compon...,[A strong component has all vertices connected...,A strong component can be reached from any oth...,"{\n ""Correct explanation"": ""In a directed gra..."


In [15]:
first_per_title.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        24 non-null     object
 1   Question     24 non-null     object
 2   Options      24 non-null     object
 3   Answer       24 non-null     object
 4   Explanation  24 non-null     object
dtypes: object(5)
memory usage: 1.1+ KB


In [16]:
import re
def parse_explanation(explanation_str):
    try:
        # Extract Wrong explanation
        correct_explanation_match = re.search(r'"Correct explanation":\s*"(.*?)"', explanation_str, re.DOTALL)
        correct_explanation = correct_explanation_match.group(1) if correct_explanation_match else ''
        
        # Extract Right explanation
        wrong_answer_match = re.search(r'"Wrong chosen answer":\s*"(.*?)"', explanation_str, re.DOTALL)
        wrong_answer = wrong_answer_match.group(1) if wrong_answer_match else ''

        wrong_explanation_match = re.search(r'"Wrong explanation":\s*"(.*?)"', explanation_str, re.DOTALL)
        wrong_explanation = wrong_explanation_match.group(1) if wrong_explanation_match else ''
        
        return pd.Series({
            'Correct Explanation': correct_explanation,
            'Wrong Answer': wrong_answer,
            'Wrong Explanation': wrong_explanation
        })
    except Exception as e:
        print(f"Error parsing explanation: {e}")
        return pd.Series({
            'Correct Explanation': '',
            'Wrong Answer': '',
            'Wrong Explanation': ''
        })

# Apply the parsing to your DataFrame
first_per_title[['Correct Explanation', 'Wrong Answer', 'Wrong Explanation']] = first_per_title['Explanation'].apply(parse_explanation)
first_per_title.head()

,Title,Question,Options,Answer,Explanation,Correct Explanation,Wrong Answer,Wrong Explanation
0,Easy Arrays,What is an array?,"[A collection of similar elements, A collectio...",A collection of similar elements,"{\n ""Correct explanation"": ""An array is a dat...",An array is a data structure that can hold mul...,A collection of different elements,An array is a versatile data structure that ca...
1,Easy Graphs,What is a minimum spanning tree?,[A tree with the minimum number of edges from ...,A tree that minimizes the total weight of edge...,"{\n ""Correct explanation"": ""A minimum spannin...",A minimum spanning tree (MST) is a subset of t...,A tree that spans all the vertices using the l...,A minimum spanning tree is a tree that spans a...
2,Easy Hashing,What is the primary purpose of using hashing i...,"[To sort data efficiently, To store data in a ...",To quickly retrieve data based on a key,"{\n ""Correct explanation"": ""Hashing is used i...",Hashing is used in data structures to quickly ...,To sort data efficiently,Hashing is used to sort data efficiently becau...
3,Easy Heaps,What data structure are heaps almost always im...,"[Linked lists, Arrays, Hash tables, Stacks]",Arrays,"{\n ""Correct explanation"": ""Heaps are almost ...",Heaps are almost always implemented as arrays ...,Linked lists,Heaps are implemented as linked lists because ...
4,Easy Linked Lists,What is the main advantage of using a linked l...,"[Constant time access to elements, Contiguous ...",Dynamic size,"{\n ""Correct explanation"": ""The main advantag...",The main advantage of using a linked list over...,Constant time access to elements,Linked lists provide constant time access to e...


Creating test cases where each question can be split into 4 different scenarios
- Case 1: Wrong answer, wrong explanation
- Case 2: Wrong answer, right explanation
- Case 3: Right answer, wrong explanation
- Case 4: Right answer, right explanation

In [17]:
def split_row(row):
    return [
        # Wrong answer, wrong explanation
        {'Title': row['Title'],
         'Question': f"""
        Given the question and the options:
        {row['Question']}
        {row['Options']}
        The correct answer is {row['Answer']}

        I chose the wrong answer {row['Wrong Answer']} as I think that {row['Wrong Explanation']}.

        Please correct any conceptual misunderstanding I have based on my explanation and explain to me why my answer is wrong.
        ## Sample format:
        Your answer is wrong. Your understanding is wrong as...
        """},

        # Wrong answer, correct explanation
        {'Title': row['Title'],
         'Question': f"""
        Given the question and the options:
        {row['Question']}
        {row['Options']}
        The correct answer is {row['Answer']}

        I chose the wrong answer {row['Wrong Answer']} as I think that {row['Correct Explanation']}.

        Please correct any conceptual misunderstanding I have based on my explanation and explain to me why my answer is wrong.
        ## Sample format:
        Your answer is wrong. Your understanding is wrong as...
        """},

        # Correct answer, wrong explanation
        {'Title': row['Title'],
         'Question': f"""
        Given the question and the options:
        {row['Question']}
        {row['Options']}
        The correct answer is {row['Answer']}

        I chose the correct answer {row['Answer']} as I think that {row['Wrong Explanation']}.

        Please correct any conceptual misunderstanding I have based on my explanation.
        ## Sample format:
        Your answer is correct. Your understanding is partially correct as...
        """},

        # Correct answer, correct explanation
        {'Title': row['Title'],
         'Question': f"""
        Given the question and the options:
        {row['Question']}
        {row['Options']}
        The correct answer is {row['Answer']}

        I chose the correct answer {row['Answer']} as I think that {row['Correct Explanation']}.

        Please correct any conceptual misunderstanding I have based on my explanation.
        ## Sample format:
        Your answer is correct. Your understanding is partially correct as...
        """},
    ]

In [18]:
expanded_data = first_per_title.apply(split_row, axis=1)

# Flatten the list of lists into a single list of dictionaries
flattened_data = [item for sublist in expanded_data for item in sublist]

test_df = pd.DataFrame(flattened_data)
test_df

,Title,Question
0,Easy Arrays,\n Given the question and the options:\...
1,Easy Arrays,\n Given the question and the options:\...
2,Easy Arrays,\n Given the question and the options:\...
3,Easy Arrays,\n Given the question and the options:\...
4,Easy Graphs,\n Given the question and the options:\...
...,...,...
91,Medium Stacks and Queues,\n Given the question and the options:\...
92,Medium Trees,\n Given the question and the options:\...
93,Medium Trees,\n Given the question and the options:\...
94,Medium Trees,\n Given the question and the options:\...


# Testing on GPT4o

Exploring how changing the chunking of the retriever documents will impact the accuracy of the model results, and finding the optimal 

- Test 1: Chunk_size = 1000, chunk_overlap = 0
- Test 2: Chunk size = 2000, chunk_overlap = 0
- Test 3: Chunk size = 2000, chunk_overlap = 100
- Test 4: Chunk size = 1000, chunk_overlap = 100

Each answer is given a score and then the scores are compared at the end

## Vector store creation script

In [58]:
# Creating function to allow for creation of vector stores with varying chunk sizes and chunk overlaps
import os
import re
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

load_dotenv()

def create_embeddings_from_pdf(pdf_path, embedding_path, chunk_size, chunk_overlap):
    # Read PDF
    pdf_reader = PdfReader(pdf_path)
    
    # Extract text from all pages
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    
    # Clean text
    text = re.sub("\s+", " ", text).strip()
    
    # Create text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    
    # Split text into documents
    documents = text_splitter.create_documents([text])
    print(f"Total documents created: {len(documents)}")
    
    # Initialize embeddings
    # embeddings = AzureOpenAIEmbeddings(
    #     azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'], 
    #     api_key=os.environ['AZURE_OPENAI_API_KEY'], 
    #     model=os.environ['TEXT_EMBEDDING_MODEL_NAME'],
    #     azure_deployment=os.environ['TEXT_EMBEDDING_DEPLOYMENT_NAME']
    # )
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    
    # Create and save vector store
    try:
        # Use create_documents method to maintain metadata
        docsearch = FAISS.from_documents(documents, embedding=embeddings)
        
        # Ensure embedding path exists
        os.makedirs(embedding_path, exist_ok=True)
        
        # Save locally
        docsearch.save_local(folder_path=embedding_path)
        print(f"Embeddings saved to {embedding_path}")
        
        return docsearch
    
    except Exception as e:
        print(f"Error creating embeddings: {e}")
        return None

## Starting the tests...
## Test 1
Chunk_size = 1000, chunk_overlap = 0

Vector store with those specifications are created first, before undergoing inference and evaluation

In [104]:
from tqdm import tqdm

# Enable progress bar for pandas
tqdm.pandas()

In [6]:
# Creating the vector store for Test 1

pdf_path = './data/(edited) DSA textbook Python.pdf'
embedding_path = './embeddings-test1'
vector_store = create_embeddings_from_pdf(pdf_path, embedding_path, 1000, 0)

if vector_store != None:
    print(f"Vector store {pdf_path} for Test 1 to {embedding_path} is a success!")

Total documents created: 1431
Embeddings saved to ./embeddings-test1
Vector store ./data/(edited) DSA textbook Python.pdf for Test 1 to ./embeddings-test1 is a success!


### One-time evaluation set-up using LangSmith (LLM-as-a-Judge)

In [23]:
# Creating the dataset (without labels/aka. reference/aka. ground truth data)
from langsmith import Client

# QA
inputs = test_df["Question"].tolist()

# outputs = 
# qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]
qa_pairs = [{"question": q} for q in test_df["Question"]]

# Create dataset
client = Client()
dataset_name = "Algotutor_MainDataset"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Testset for optimising retrievers",
)
client.create_examples(
    inputs=[{"question": q} for q in inputs],
    #outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id,
)

In [24]:
### RAG
import os
import openai
from openai import AzureOpenAI
from langsmith import traceable
from langsmith.wrappers import wrap_openai

class RagBot:
    
    def __init__(self, retriever, model: str = 'gpt-4o'):
        self._retriever = retriever
        # Wrapping the client instruments the LLM
        # self._client = wrap_openai(AzureOpenAI(
        #     azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
        #     api_key=os.environ['AZURE_OPENAI_API_KEY'],
        #     api_version=os.environ['AZURE_OPENAI_API_VERSION']
        # ))
        self._client = wrap_openai(openai.Client())
        self._model = model

    @traceable()
    def retrieve_docs(self, question):
        return self._retriever.similarity_search(question, k=2)

    @traceable()
    def get_answer(self, question: str):
        similar = self.retrieve_docs(question)
        response = self._client.chat.completions.create(
            model=self._model,
            messages=[
                {
                    "role": "system",
                    "content": "You are a teaching assistant. Your task is to answer student query about Data Structures and Algorithms in Python course. If user asks any query beyond data structures and algorithms, tell the user you are not an expert of the topic."
                    "Answer the question based only on the following context:\n\n"
                    f"Context:\n\n{similar}",
                },
                {"role": "user", "content": question},
            ],
            temperature=0
        )

        # Evaluators will expect "answer" and "contexts"
        return {
            "answer": response.choices[0].message.content,
            "contexts": [str(doc) for doc in similar],
        }

In [50]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate
import textwrap

# Checking whether the answer is accurate to the docs retrieved
# answer_accuracy_evaluator = LangChainStringEvaluator(
#   "labeled_score_string",
#   config={
#       "criteria": {
#           "accuracy": textwrap.dedent("""Is the Assistant's Answer grounded in the Ground Truth documentation? A score of [[1]] means that the
#             Assistant answer contains is not at all based upon / grounded in the Groun Truth documentation. A score of [[5]] means 
#             that the Assistant answer contains some information (e.g., a hallucination) that is not captured in the Ground Truth 
#             documentation. A score of [[10]] means that the Assistant answer is fully based upon the in the Ground Truth documentation."""
#           )
#       },
#       # If you want the score to be saved on a scale from 0 to 1
#       "normalize_by": 10,
#   },
#   prepare_data=lambda run, example: {
#         "prediction": run.outputs["answer"],
#         "reference": run.outputs["contexts"],
#         "input": example.inputs["question"],
#     },
# )

# Checking whether the retrieved documents are relevant to question
docs_relevance_evaluator = LangChainStringEvaluator(
    "score_string",
    config={
        "criteria": {
            "document_relevance": textwrap.dedent(
                """The response is a set of documents retrieved from a vectorstore. The input is a question
            used for retrieval. You will score whether the Assistant's response (retrieved docs) is relevant to the Ground Truth 
            question. A score of [[1]] means that none of the Assistant's response documents contain information useful in answering or addressing the user's input.
            A score of [[5]] means that the Assistant answer contains some relevant documents that can at least partially answer the user's question or input. 
            A score of [[10]] means that the user input can be fully answered using the content in the first retrieved doc(s)."""
            )
        },
        # If you want the score to be saved on a scale from 0 to 1
        "normalize_by": 10
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["contexts"],
        "input": example.inputs["question"],
    },
)

This chain was only tested with GPT-4. Performance may be significantly worse with other models.


### Evaluation on Test 1 using first retriever

In [25]:
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS

embedding_model = AzureOpenAIEmbeddings(azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'], 
                                   api_key=os.environ['AZURE_OPENAI_API_KEY'], 
                                   model=os.environ['TEXT_EMBEDDING_MODEL_NAME'],
                                   azure_deployment=os.environ['TEXT_EMBEDDING_DEPLOYMENT_NAME'])

docsearch = FAISS.load_local("./embeddings-test1", embeddings=embedding_model, allow_dangerous_deserialization=True)

rag_bot = RagBot(docsearch)

def predict_rag_answer_with_context(example: dict):
    """Use this for evaluation of retrieved documents and hallucinations"""
    response = rag_bot.get_answer(example["question"])
    return {"answer": response["answer"], "contexts": response["contexts"]}

In [28]:
# Dataset has been split on LangSmith interface into 10 splits
# This is to bypass the token restriction on the OpenAI API calls

from time import sleep

for setnumber in range(1, 11):
    dataset_name = "Algotutor_MainDataset"
    experiment_results = evaluate(
        predict_rag_answer_with_context,
        data=client.list_examples(dataset_name=dataset_name, splits=[f"set{setnumber}"]),
        evaluators=[docs_relevance_evaluator],
        experiment_prefix=f"rag-chunk1000-overlap0-doc-relevance-{setnumber}",
        # Any experiment metadata can be specified here
        metadata={
            "variant": "chunk_size=1000, chunk_overlap=0",
        },
    )
    print(f"Datasplit set{setnumber} evaluation completed")

    # Sleep in order to bypass the minute rate limit on the token calls
    if setnumber < 10:
        sleep(60)

/root/algotutor-fyp/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'rag-chunk1000-overlap0-doc-relevance-1-1aab139f' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/26ec1cbb-8079-4a3a-bd80-1141f080e39f/compare?selectedSessions=853581aa-555e-4eca-a322-25194d6372e3




10it [00:13,  1.34s/it]


Datasplit set1 evaluation completed
View the evaluation results for experiment: 'rag-chunk1000-overlap0-doc-relevance-2-ee11b364' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/26ec1cbb-8079-4a3a-bd80-1141f080e39f/compare?selectedSessions=c27acea5-7f7e-40c8-8ec4-4a5abca8556f




10it [00:44,  4.47s/it]


...

Datasplit set8 evaluation completed
View the evaluation results for experiment: 'rag-chunk1000-overlap0-doc-relevance-9-ef53b19a' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/26ec1cbb-8079-4a3a-bd80-1141f080e39f/compare?selectedSessions=fe57c17a-961f-47a9-85d1-07dff0d6cc5b




10it [00:16,  1.64s/it]


Datasplit set9 evaluation completed
View the evaluation results for experiment: 'rag-chunk1000-overlap0-doc-relevance-10-0aad8cb5' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/26ec1cbb-8079-4a3a-bd80-1141f080e39f/compare?selectedSessions=b03f93b9-73c9-49bc-bead-60e816c537b2




6it [00:12,  2.05s/it]


Datasplit set10 evaluation completed


In [30]:
project_names = ['rag-chunk1000-overlap0-doc-relevance-1-1aab139f', 'rag-chunk1000-overlap0-doc-relevance-2-ee11b364', 'rag-chunk1000-overlap0-doc-relevance-3-b3775221', 'rag-chunk1000-overlap0-doc-relevance-4-154aa696', 'rag-chunk1000-overlap0-doc-relevance-5-f265f1b9', 'rag-chunk1000-overlap0-doc-relevance-6-1b2e4b31', 'rag-chunk1000-overlap0-doc-relevance-7-b582aef2', 'rag-chunk1000-overlap0-doc-relevance-8-7d496f3f', 'rag-chunk1000-overlap0-doc-relevance-9-ef53b19a', 'rag-chunk1000-overlap0-doc-relevance-10-0aad8cb5']

#for projectidx in range(len(project_names)):

all_dfs = [client.get_test_results(project_name=project_names[projectidx]) for projectidx in range(len(project_names))]
combined_df = pd.concat(all_dfs, ignore_index=True)

combined_df.head()

/tmp/ipykernel_678/4090098211.py:5: UserWarning: Function get_test_results is in beta.
  all_dfs = [client.get_test_results(project_name=project_names[projectidx]) for projectidx in range(len(project_names))]
/tmp/ipykernel_678/4090098211.py:5: UserWarning: Function get_test_results is in beta.
  all_dfs = [client.get_test_results(project_name=project_names[projectidx]) for projectidx in range(len(project_names))]
/tmp/ipykernel_678/4090098211.py:5: UserWarning: Function get_test_results is in beta.
  all_dfs = [client.get_test_results(project_name=project_names[projectidx]) for projectidx in range(len(project_names))]
/tmp/ipykernel_678/4090098211.py:5: UserWarning: Function get_test_results is in beta.
  all_dfs = [client.get_test_results(project_name=project_names[projectidx]) for projectidx in range(len(project_names))]
/tmp/ipykernel_678/4090098211.py:5: UserWarning: Function get_test_results is in beta.
  all_dfs = [client.get_test_results(project_name=project_names[projectidx]) 

,outputs.answer,outputs.contexts,execution_time,error,id,feedback.score_string:document_relevance,input.example.question
0,Your answer is incorrect. Your understanding o...,[page_content='configuration of the Queue Visu...,4.325442,None,25e8b861-0d1f-40d8-8e68-b69fb89ae9ba,0.5,\n Given the question and the options:\...
1,Your answer is correct. Your understanding is ...,[page_content='configuration of the Queue Visu...,3.903861,None,2c6860b7-53cb-47df-9a33-975acf198691,0.5,\n Given the question and the options:\...
2,Your answer is correct. Your understanding is ...,"[page_content='you can see, the time required ...",4.223810,None,ec0b6271-e169-4056-b798-f79e8b49ebf7,1.0,\n Given the question and the options:\...
3,Your answer is correct. Your understanding is ...,[page_content='trickiest parts isremembering w...,3.491941,None,b5f5fda2-49fd-4d24-90af-05a951f43166,0.5,\n Given the question and the options:\...
4,Your answer is wrong. Your understanding is co...,[page_content='configuration of the Queue Visu...,5.009560,None,9f8e1409-e856-4195-b487-f8d922819f4d,0.5,\n Given the question and the options:\...


In [32]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 7 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   outputs.answer                            96 non-null     object 
 1   outputs.contexts                          96 non-null     object 
 2   execution_time                            96 non-null     float64
 3   error                                     0 non-null      object 
 4   id                                        96 non-null     object 
 5   feedback.score_string:document_relevance  96 non-null     float64
 6   input.example.question                    96 non-null     object 
dtypes: float64(2), object(5)
memory usage: 5.4+ KB


In [36]:
# Merge the two DataFrames on the matching columns
merge_df = pd.merge(
    combined_df, test_df,
    left_on="input.example.question",
    right_on="Question",
    how="inner"  # Use "inner" to include only matching rows
)

desired_column_order = [
    "id",
    "Title",
    "input.example.question",
    "outputs.answer",
    "outputs.contexts",
    "feedback.score_string:document_relevance",
    "execution_time",
    "error"
]

# Reorder the DataFrame columns
results_df = merge_df[desired_column_order]

new_column_names = {
    "id": "ID",
    "Title": "Title",
    "input.example.question": "Question",  
    "outputs.answer": "Model Answer",                  
    "outputs.contexts": "Retrieved Context",
    "feedback.score_string:document_relevance": "Relevance Score",
    "execution_time": "Execution Time",
    "error": "Error"
}

results_df.rename(columns=new_column_names, inplace=True)

/tmp/ipykernel_678/3690702879.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df.rename(columns=new_column_names, inplace=True)


In [37]:
results_df.head()

,ID,Title,Question,Model Answer,Retrieved Context,Relevance Score,Execution Time,Error
0,25e8b861-0d1f-40d8-8e68-b69fb89ae9ba,Medium Stacks and Queues,\n Given the question and the options:\...,Your answer is incorrect. Your understanding o...,[page_content='configuration of the Queue Visu...,0.5,4.325442,None
1,2c6860b7-53cb-47df-9a33-975acf198691,Medium Stacks and Queues,\n Given the question and the options:\...,Your answer is correct. Your understanding is ...,[page_content='configuration of the Queue Visu...,0.5,3.903861,None
2,ec0b6271-e169-4056-b798-f79e8b49ebf7,Medium Trees,\n Given the question and the options:\...,Your answer is correct. Your understanding is ...,"[page_content='you can see, the time required ...",1.0,4.223810,None
3,b5f5fda2-49fd-4d24-90af-05a951f43166,Medium Recursion,\n Given the question and the options:\...,Your answer is correct. Your understanding is ...,[page_content='trickiest parts isremembering w...,0.5,3.491941,None
4,9f8e1409-e856-4195-b487-f8d922819f4d,Medium Stacks and Queues,\n Given the question and the options:\...,Your answer is wrong. Your understanding is co...,[page_content='configuration of the Queue Visu...,0.5,5.009560,None


In [38]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 96 non-null     object 
 1   Title              96 non-null     object 
 2   Question           96 non-null     object 
 3   Model Answer       96 non-null     object 
 4   Retrieved Context  96 non-null     object 
 5   Relevance Score    96 non-null     float64
 6   Execution Time     96 non-null     float64
 7   Error              0 non-null      object 
dtypes: float64(2), object(6)
memory usage: 6.1+ KB


### Score for Test 1

Calculating the score by taking the mean of all the relevance scores for each question/retrieved context pair

In [40]:
print(f"Score for Test 1 retriever: {results_df['Relevance Score'].mean()}")

Score for Test 1 retriever: 0.7145833333333332


## Test 2
Chunk_size = 2000, chunk_overlap = 0

Vector store with those specifications are created first, before undergoing inference and evaluation

In [46]:
# Creating the vector store for Test 1

pdf_path = './data/(edited) DSA textbook Python.pdf'
embedding_path = './embeddings-test2'
vector_store = create_embeddings_from_pdf(pdf_path, embedding_path, 2000, 0)

if vector_store != None:
    print(f"Vector store {pdf_path} for Test 2 to {embedding_path} is a success!")

Total documents created: 715
Embeddings saved to ./embeddings-test2
Vector store ./data/(edited) DSA textbook Python.pdf for Test 2 to ./embeddings-test2 is a success!


### Evaluation on Test 2 using second retriever

In [48]:
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS

embedding_model = AzureOpenAIEmbeddings(azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'], 
                                   api_key=os.environ['AZURE_OPENAI_API_KEY'], 
                                   model=os.environ['TEXT_EMBEDDING_MODEL_NAME'],
                                   azure_deployment=os.environ['TEXT_EMBEDDING_DEPLOYMENT_NAME'])

docsearch = FAISS.load_local("./embeddings-test2", embeddings=embedding_model, allow_dangerous_deserialization=True)

rag_bot = RagBot(docsearch)

def predict_rag_answer_with_context(example: dict):
    """Use this for evaluation of retrieved documents and hallucinations"""
    response = rag_bot.get_answer(example["question"])
    return {"answer": response["answer"], "contexts": response["contexts"]}

In [52]:
# Dataset has been split on LangSmith interface into 20 splits
# This is to bypass the token restriction on the OpenAI API calls

from time import sleep

for setnumber in range(1, 21):
    dataset_name = "Algotutor_Dataset_20split"
    experiment_results = evaluate(
        predict_rag_answer_with_context,
        data=client.list_examples(dataset_name=dataset_name, splits=[f"set{setnumber}"]),
        evaluators=[docs_relevance_evaluator],
        experiment_prefix=f"rag-chunk2000-overlap0-doc-relevance-{setnumber}",
        # Any experiment metadata can be specified here
        metadata={
            "variant": "chunk_size=2000, chunk_overlap=0",
        },
    )
    print(f"Datasplit set{setnumber} evaluation completed")

    # Sleep in order to bypass the minute rate limit on the token calls
    if setnumber < 20:
        sleep(30)

View the evaluation results for experiment: 'rag-chunk2000-overlap0-doc-relevance-1-f68e44f4' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/2ffa90d0-c4c7-4365-b7f2-db681b2d2c8a/compare?selectedSessions=c67bb198-6bc0-45b2-b254-7a3a559fd558




5it [00:15,  3.03s/it]


Datasplit set1 evaluation completed
View the evaluation results for experiment: 'rag-chunk2000-overlap0-doc-relevance-2-8c3ce496' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/2ffa90d0-c4c7-4365-b7f2-db681b2d2c8a/compare?selectedSessions=14aca239-b155-4c9c-9a37-e213553a5866




5it [00:11,  2.32s/it]


Datasplit set2 evaluation completed
View the evaluation results for experiment: 'rag-chunk2000-overlap0-doc-relevance-3-ef140a31' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/2ffa90d0-c4c7-4365-b7f2-db681b2d2c8a/compare?selectedSessions=09cff7d1-20bb-4bcd-95b9-f49211700f73




...

Datasplit set18 evaluation completed
View the evaluation results for experiment: 'rag-chunk2000-overlap0-doc-relevance-19-43de23fd' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/2ffa90d0-c4c7-4365-b7f2-db681b2d2c8a/compare?selectedSessions=fff93734-4565-4eab-9d32-06e9af003f59




5it [00:13,  2.73s/it]


Datasplit set19 evaluation completed
View the evaluation results for experiment: 'rag-chunk2000-overlap0-doc-relevance-20-05145500' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/2ffa90d0-c4c7-4365-b7f2-db681b2d2c8a/compare?selectedSessions=f7242759-6acc-4ee6-8bf1-b9f1d72041ec




3it [00:11,  3.75s/it]

Datasplit set20 evaluation completed


In [53]:
project_names_test2 = ['rag-chunk2000-overlap0-doc-relevance-1-f68e44f4', 'rag-chunk2000-overlap0-doc-relevance-2-8c3ce496', 'rag-chunk2000-overlap0-doc-relevance-3-ef140a31', 'rag-chunk2000-overlap0-doc-relevance-4-ae6d3aee', 'rag-chunk2000-overlap0-doc-relevance-5-4560bbce', 'rag-chunk2000-overlap0-doc-relevance-6-57993c74', 'rag-chunk2000-overlap0-doc-relevance-7-10cae52f', 'rag-chunk2000-overlap0-doc-relevance-8-e46801b5', 'rag-chunk2000-overlap0-doc-relevance-9-02eef999', 'rag-chunk2000-overlap0-doc-relevance-10-15e48fb8', 'rag-chunk2000-overlap0-doc-relevance-11-9b60f173', 'rag-chunk2000-overlap0-doc-relevance-12-d20e1afa', 'rag-chunk2000-overlap0-doc-relevance-13-af0a3865', 'rag-chunk2000-overlap0-doc-relevance-14-5f083228', 'rag-chunk2000-overlap0-doc-relevance-15-74f68302', 'rag-chunk2000-overlap0-doc-relevance-16-c09df657','rag-chunk2000-overlap0-doc-relevance-17-0ccdea9e', 'rag-chunk2000-overlap0-doc-relevance-18-af9cb8ce', 'rag-chunk2000-overlap0-doc-relevance-19-43de23fd', 'rag-chunk2000-overlap0-doc-relevance-20-05145500']

all_dfs_test2 = [client.get_test_results(project_name=project_names_test2[projectidx]) for projectidx in range(len(project_names_test2))]
combined_df_test2 = pd.concat(all_dfs_test2, ignore_index=True)

# Merge the two DataFrames on the matching columns
merge_df_test2 = pd.merge(
    combined_df_test2, test_df,
    left_on="input.example.question",
    right_on="Question",
    how="inner"  # Use "inner" to include only matching rows
)

# Reorder the DataFrame columns
results_df_test2 = merge_df_test2[desired_column_order]

results_df_test2.rename(columns=new_column_names, inplace=True)

/tmp/ipykernel_678/690879135.py:3: UserWarning: Function get_test_results is in beta.
  all_dfs_test2 = [client.get_test_results(project_name=project_names_test2[projectidx]) for projectidx in range(len(project_names_test2))]
/tmp/ipykernel_678/690879135.py:3: UserWarning: Function get_test_results is in beta.
  all_dfs_test2 = [client.get_test_results(project_name=project_names_test2[projectidx]) for projectidx in range(len(project_names_test2))]
/tmp/ipykernel_678/690879135.py:3: UserWarning: Function get_test_results is in beta.
  all_dfs_test2 = [client.get_test_results(project_name=project_names_test2[projectidx]) for projectidx in range(len(project_names_test2))]
/tmp/ipykernel_678/690879135.py:3: UserWarning: Function get_test_results is in beta.
  all_dfs_test2 = [client.get_test_results(project_name=project_names_test2[projectidx]) for projectidx in range(len(project_names_test2))]
/tmp/ipykernel_678/690879135.py:3: UserWarning: Function get_test_results is in beta.
  all_dfs_

Score for Test 2 retriever: 0.671875


/tmp/ipykernel_678/690879135.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df_test2.rename(columns=new_column_names, inplace=True)


### Score for Test 2

In [54]:
print(f"Score for Test 2 retriever: {results_df_test2['Relevance Score'].mean()}")

Score for Test 2 retriever: 0.671875


## Test 3
Chunk_size = 2000, chunk_overlap = 100

Vector store with those specifications are created first, before undergoing inference and evaluation

In [59]:
# Creating the vector store for Test 1

pdf_path = './data/(edited) DSA textbook Python.pdf'
embedding_path = './embeddings-test3'
vector_store = create_embeddings_from_pdf(pdf_path, embedding_path, 2000, 100)

if vector_store != None:
    print(f"Vector store {pdf_path} for Test 3 to {embedding_path} is a success!")

Total documents created: 751
Embeddings saved to ./embeddings-test3
Vector store ./data/(edited) DSA textbook Python.pdf for Test 3 to ./embeddings-test3 is a success!


### Evaluation on Test 3 using third retriever

In [60]:
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS

embedding_model = AzureOpenAIEmbeddings(azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'], 
                                   api_key=os.environ['AZURE_OPENAI_API_KEY'], 
                                   model=os.environ['TEXT_EMBEDDING_MODEL_NAME'],
                                   azure_deployment=os.environ['TEXT_EMBEDDING_DEPLOYMENT_NAME'])

docsearch = FAISS.load_local("./embeddings-test3", embeddings=embedding_model, allow_dangerous_deserialization=True)

rag_bot = RagBot(docsearch)

def predict_rag_answer_with_context(example: dict):
    """Use this for evaluation of retrieved documents and hallucinations"""
    response = rag_bot.get_answer(example["question"])
    return {"answer": response["answer"], "contexts": response["contexts"]}

In [61]:
# Dataset has been split on LangSmith interface into 20 splits
# This is to bypass the token restriction on the OpenAI API calls

from time import sleep

for setnumber in range(1, 21):
    dataset_name = "Algotutor_Dataset_20split"
    experiment_results = evaluate(
        predict_rag_answer_with_context,
        data=client.list_examples(dataset_name=dataset_name, splits=[f"set{setnumber}"]),
        evaluators=[docs_relevance_evaluator],
        experiment_prefix=f"rag-chunk2000-overlap100-doc-relevance-{setnumber}",
        # Any experiment metadata can be specified here
        metadata={
            "variant": "chunk_size=2000, chunk_overlap=100",
        },
    )
    print(f"Datasplit set{setnumber} evaluation completed")

    # Sleep in order to bypass the minute rate limit on the token calls
    if setnumber < 20:
        sleep(30)

View the evaluation results for experiment: 'rag-chunk2000-overlap100-doc-relevance-1-db735c22' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/2ffa90d0-c4c7-4365-b7f2-db681b2d2c8a/compare?selectedSessions=ea4af3ec-de2c-4f8a-8cf2-de5ede7ca681




5it [00:13,  2.79s/it]


Datasplit set1 evaluation completed
View the evaluation results for experiment: 'rag-chunk2000-overlap100-doc-relevance-2-e33de573' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/2ffa90d0-c4c7-4365-b7f2-db681b2d2c8a/compare?selectedSessions=41d07cba-fd39-483c-8ad7-ffa2cd58fae0




5it [00:15,  3.14s/it]


Datasplit set2 evaluation completed
View the evaluation results for experiment: 'rag-chunk2000-overlap100-doc-relevance-3-777f0047' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/2ffa90d0-c4c7-4365-b7f2-db681b2d2c8a/compare?selectedSessions=1800b542-3590-4d6e-a7cd-ff81ffe2728d




...

Datasplit set18 evaluation completed
View the evaluation results for experiment: 'rag-chunk2000-overlap100-doc-relevance-19-7c6a558a' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/2ffa90d0-c4c7-4365-b7f2-db681b2d2c8a/compare?selectedSessions=cf170818-8bff-480f-b1a8-a00f834d8ee2




5it [00:10,  2.12s/it]


Datasplit set19 evaluation completed
View the evaluation results for experiment: 'rag-chunk2000-overlap100-doc-relevance-20-94845d60' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/2ffa90d0-c4c7-4365-b7f2-db681b2d2c8a/compare?selectedSessions=5527c247-30dd-4a07-bd41-27ee14cb0405




3it [00:11,  3.96s/it]


Datasplit set20 evaluation completed


In [64]:
project_names_test3= ['rag-chunk2000-overlap100-doc-relevance-1-db735c22', 'rag-chunk2000-overlap100-doc-relevance-2-e33de573', 'rag-chunk2000-overlap100-doc-relevance-3-777f0047', 'rag-chunk2000-overlap100-doc-relevance-4-2cf804c5', 'rag-chunk2000-overlap100-doc-relevance-5-63f09e84', 'rag-chunk2000-overlap100-doc-relevance-6-b79c1e51', 'rag-chunk2000-overlap100-doc-relevance-7-9163e970', 'rag-chunk2000-overlap100-doc-relevance-8-958562b8', 'rag-chunk2000-overlap100-doc-relevance-9-4091d4a2', 'rag-chunk2000-overlap100-doc-relevance-10-232412a1', 'rag-chunk2000-overlap100-doc-relevance-11-cdf454c6', 'rag-chunk2000-overlap100-doc-relevance-12-fa301475', 'rag-chunk2000-overlap100-doc-relevance-13-217659a1', 'rag-chunk2000-overlap100-doc-relevance-14-7781698d', 'rag-chunk2000-overlap100-doc-relevance-15-df4d8554', 'rag-chunk2000-overlap100-doc-relevance-16-627ac9e4', 'rag-chunk2000-overlap100-doc-relevance-17-9cdaf4a1', 'rag-chunk2000-overlap100-doc-relevance-18-c717e573', 'rag-chunk2000-overlap100-doc-relevance-19-7c6a558a', 'rag-chunk2000-overlap100-doc-relevance-20-94845d60']

all_dfs_test3 = [client.get_test_results(project_name=project_names_test3[projectidx]) for projectidx in range(len(project_names_test3))]
combined_df_test3 = pd.concat(all_dfs_test3, ignore_index=True)

# Merge the two DataFrames on the matching columns
merge_df_test3 = pd.merge(
    combined_df_test3, test_df,
    left_on="input.example.question",
    right_on="Question",
    how="inner"  # Use "inner" to include only matching rows
)

# Reorder the DataFrame columns
results_df_test3 = merge_df_test3[desired_column_order]

results_df_test3.rename(columns=new_column_names, inplace=True)

/tmp/ipykernel_678/92923380.py:3: UserWarning: Function get_test_results is in beta.
  all_dfs_test3 = [client.get_test_results(project_name=project_names_test3[projectidx]) for projectidx in range(len(project_names_test3))]
/tmp/ipykernel_678/92923380.py:3: UserWarning: Function get_test_results is in beta.
  all_dfs_test3 = [client.get_test_results(project_name=project_names_test3[projectidx]) for projectidx in range(len(project_names_test3))]
/tmp/ipykernel_678/92923380.py:3: UserWarning: Function get_test_results is in beta.
  all_dfs_test3 = [client.get_test_results(project_name=project_names_test3[projectidx]) for projectidx in range(len(project_names_test3))]
/tmp/ipykernel_678/92923380.py:3: UserWarning: Function get_test_results is in beta.
  all_dfs_test3 = [client.get_test_results(project_name=project_names_test3[projectidx]) for projectidx in range(len(project_names_test3))]
/tmp/ipykernel_678/92923380.py:3: UserWarning: Function get_test_results is in beta.
  all_dfs_test3

### Score for Test 3

In [65]:
print(f"Score for Test 3 retriever: {results_df_test3['Relevance Score'].mean()}")

Score for Test 3 retriever: 0.6479166666666667


## Test 4
Chunk_size = 1000, chunk_overlap = 100

Vector store with those specifications are created first, before undergoing inference and evaluation

In [66]:
# Creating the vector store for Test 1

pdf_path = './data/(edited) DSA textbook Python.pdf'
embedding_path = './embeddings-test4'
vector_store = create_embeddings_from_pdf(pdf_path, embedding_path, 1000, 100)

if vector_store != None:
    print(f"Vector store {pdf_path} for Test 4 to {embedding_path} is a success!")

Total documents created: 1585
Embeddings saved to ./embeddings-test4
Vector store ./data/(edited) DSA textbook Python.pdf for Test 4 to ./embeddings-test4 is a success!


### Evaluation on Test 4 using fourth retriever

In [67]:
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS

embedding_model = AzureOpenAIEmbeddings(azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'], 
                                   api_key=os.environ['AZURE_OPENAI_API_KEY'], 
                                   model=os.environ['TEXT_EMBEDDING_MODEL_NAME'],
                                   azure_deployment=os.environ['TEXT_EMBEDDING_DEPLOYMENT_NAME'])

docsearch = FAISS.load_local("./embeddings-test4", embeddings=embedding_model, allow_dangerous_deserialization=True)

rag_bot = RagBot(docsearch)

def predict_rag_answer_with_context(example: dict):
    """Use this for evaluation of retrieved documents and hallucinations"""
    response = rag_bot.get_answer(example["question"])
    return {"answer": response["answer"], "contexts": response["contexts"]}

In [68]:
# Dataset has been split on LangSmith interface into 20 splits
# This is to bypass the token restriction on the OpenAI API calls

from time import sleep

for setnumber in range(1, 21):
    dataset_name = "Algotutor_Dataset_20split"
    experiment_results = evaluate(
        predict_rag_answer_with_context,
        data=client.list_examples(dataset_name=dataset_name, splits=[f"set{setnumber}"]),
        evaluators=[docs_relevance_evaluator],
        experiment_prefix=f"rag-chunk1000-overlap100-doc-relevance-{setnumber}",
        # Any experiment metadata can be specified here
        metadata={
            "variant": "chunk_size=1000, chunk_overlap=100",
        },
    )
    print(f"Datasplit set{setnumber} evaluation completed")

    # Sleep in order to bypass the minute rate limit on the token calls
    if setnumber < 20:
        sleep(30)

View the evaluation results for experiment: 'rag-chunk1000-overlap100-doc-relevance-1-39242bf8' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/2ffa90d0-c4c7-4365-b7f2-db681b2d2c8a/compare?selectedSessions=5b070019-0bde-48cc-a008-3d2f72641467




5it [00:13,  2.74s/it]


Datasplit set1 evaluation completed
View the evaluation results for experiment: 'rag-chunk1000-overlap100-doc-relevance-2-ffdc9b39' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/2ffa90d0-c4c7-4365-b7f2-db681b2d2c8a/compare?selectedSessions=4946b109-a453-48e1-8c26-c9bc8369c59b




5it [00:14,  2.81s/it]


Datasplit set2 evaluation completed
View the evaluation results for experiment: 'rag-chunk1000-overlap100-doc-relevance-3-2029bff0' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/2ffa90d0-c4c7-4365-b7f2-db681b2d2c8a/compare?selectedSessions=4c8e8c2c-d9f3-470a-858f-b19c4940e1f1




...

Datasplit set18 evaluation completed
View the evaluation results for experiment: 'rag-chunk1000-overlap100-doc-relevance-19-202cd5df' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/2ffa90d0-c4c7-4365-b7f2-db681b2d2c8a/compare?selectedSessions=33bb1b67-3b2b-4477-b29e-985b25f5788d




5it [00:13,  2.68s/it]


Datasplit set19 evaluation completed
View the evaluation results for experiment: 'rag-chunk1000-overlap100-doc-relevance-20-2c5009fe' at:
https://smith.langchain.com/o/65a167b9-d4dd-594a-9fef-747869e69109/datasets/2ffa90d0-c4c7-4365-b7f2-db681b2d2c8a/compare?selectedSessions=38bdda40-c87d-4c55-8a2a-492484e195b0




3it [00:12,  4.06s/it]

Datasplit set20 evaluation completed


In [69]:
project_names_test4= ['rag-chunk1000-overlap100-doc-relevance-1-39242bf8', 'rag-chunk1000-overlap100-doc-relevance-2-ffdc9b39', 'rag-chunk1000-overlap100-doc-relevance-3-2029bff0', 'rag-chunk1000-overlap100-doc-relevance-4-5028a9fa', 'rag-chunk1000-overlap100-doc-relevance-5-f19d72c9', 'rag-chunk1000-overlap100-doc-relevance-6-fb109294', 'rag-chunk1000-overlap100-doc-relevance-7-a0350f08', 'rag-chunk1000-overlap100-doc-relevance-8-dadf82ec', 'rag-chunk1000-overlap100-doc-relevance-9-866a1bd6', 'rag-chunk1000-overlap100-doc-relevance-10-a9cf247d', 'rag-chunk1000-overlap100-doc-relevance-11-39fb5df0', 'rag-chunk1000-overlap100-doc-relevance-12-f82155a2', 'rag-chunk1000-overlap100-doc-relevance-13-838b4ef8', 'rag-chunk1000-overlap100-doc-relevance-14-2ec3c3c0', 'rag-chunk1000-overlap100-doc-relevance-15-28119639', 'rag-chunk1000-overlap100-doc-relevance-16-dc20cbe5', 'rag-chunk1000-overlap100-doc-relevance-17-f0ff6a90', 'rag-chunk1000-overlap100-doc-relevance-18-08aaafb9', 'rag-chunk1000-overlap100-doc-relevance-19-202cd5df', 'rag-chunk1000-overlap100-doc-relevance-20-2c5009fe']

all_dfs_test4 = [client.get_test_results(project_name=project_names_test4[projectidx]) for projectidx in range(len(project_names_test4))]
combined_df_test4 = pd.concat(all_dfs_test4, ignore_index=True)

# Merge the two DataFrames on the matching columns
merge_df_test4 = pd.merge(
    combined_df_test4, test_df,
    left_on="input.example.question",
    right_on="Question",
    how="inner"  # Use "inner" to include only matching rows
)

# Reorder the DataFrame columns
results_df_test4 = merge_df_test4[desired_column_order]

results_df_test4.rename(columns=new_column_names, inplace=True)

/tmp/ipykernel_678/4238294787.py:3: UserWarning: Function get_test_results is in beta.
  all_dfs_test4 = [client.get_test_results(project_name=project_names_test4[projectidx]) for projectidx in range(len(project_names_test4))]
/tmp/ipykernel_678/4238294787.py:3: UserWarning: Function get_test_results is in beta.
  all_dfs_test4 = [client.get_test_results(project_name=project_names_test4[projectidx]) for projectidx in range(len(project_names_test4))]
/tmp/ipykernel_678/4238294787.py:3: UserWarning: Function get_test_results is in beta.
  all_dfs_test4 = [client.get_test_results(project_name=project_names_test4[projectidx]) for projectidx in range(len(project_names_test4))]
/tmp/ipykernel_678/4238294787.py:3: UserWarning: Function get_test_results is in beta.
  all_dfs_test4 = [client.get_test_results(project_name=project_names_test4[projectidx]) for projectidx in range(len(project_names_test4))]
/tmp/ipykernel_678/4238294787.py:3: UserWarning: Function get_test_results is in beta.
  all

### Score for Test 4

In [71]:
print(f"Score for Test 4 retriever: {results_df_test4['Relevance Score'].mean()}")

Score for Test 4 retriever: 0.7177083333333334


# Conclusion

Recap on the retriever's specifications for each test:
- Test 1: Chunk_size = 1000, chunk_overlap = 0
- Test 2: Chunk size = 2000, chunk_overlap = 0
- Test 3: Chunk size = 2000, chunk_overlap = 100
- Test 4: Chunk size = 1000, chunk_overlap = 100

### Final Table Summary of all the retriever's relevance scores

In [80]:
scores = [
    results_df['Relevance Score'].mean(),
    results_df_test2['Relevance Score'].mean(),
    results_df_test3['Relevance Score'].mean(),
    results_df_test4['Relevance Score'].mean(),
]

final_df = pd.DataFrame({
    'Test': [1, 2, 3, 4],
    'Score': scores,
    'Chunk_Size': [1000, 2000, 2000, 1000],
    'Chunk_Overlap': [0, 0, 100, 100]
}, index=None)

final_df

,Test,Score,Chunk_Size,Chunk_Overlap
0,1,0.714583,1000,0
1,2,0.671875,2000,0
2,3,0.647917,2000,100
3,4,0.717708,1000,100


In [79]:
# Getting the excel file results

results_df['Test'] = 1
results_df_test2['Test'] = 2
results_df_test3['Test'] = 3
results_df_test4['Test'] = 4

all_results_df = pd.concat([results_df, results_df_test2, results_df_test3, results_df_test4], ignore_index=True)

# Saving the concatenated DataFrame to an Excel file
all_results_df.to_excel('concatenated_evaluation_results.xlsx', index=False)